# 变分量子算法
Written by MindQuantum Team
<br>

1. 背景介绍
2. 变分量子算法的要素构成
   1.1 量子比特<br>
   2.1 参数化量子门<br>
   2.2 波函数拟设<br>
   2.3 损失函数<br>
   2.4 求导规则<br>
   2.5 优化步骤<br>
3. 应用领域<br>
4. 作业<br><br>

## 1. 背景介绍

![intro](images/intro.png)

### NISQ阶段的有效量子算法：变分量子算法VQA

![vqe](images/vqa.png)

变分量子算法是结合量子算法和经典优化算法的量子算法，其基本步骤如下：

1. 设计参数化量子线路
2. 根据观察量定义好损失函数
3. 根据损失函数，利用经典优化器更新参数

### 应用场景

1. 哈密顿量的基态能量求解
2. 监督学习/非监督学习

## 2. 变分量子算法的要素构成

### 2.1 参数化量子门

非参数化量子门通常有：`X`，`Y`，`Z`，`H`，`CNOT`等。
参数化量子门：`RX`，`RY`，`RZ`等。

例如绕`X`，`Y`，`Z`轴旋转$\theta$角的旋转门：

$$RX(\theta) = e^{-i\theta X/2}=\cos(\theta/2)I-i\sin(\theta/2)X$$
$$RY(\theta) = e^{-i\theta Y/2}=\cos(\theta/2)I-i\sin(\theta/2)Y$$
$$RZ(\theta) = e^{-i\theta Z/2}=\cos(\theta/2)I-i\sin(\theta/2)Z$$

更一般的：

$$e^{-iθ (\vec{v} \cdot \vec{\sigma})}=\cos(\theta/2)I-i\sin(\theta/2)\vec{v}\cdot \vec{\sigma}$$

其中$\vec{v}$是任意的三维单位矢量，$\vec{\sigma}=(\sigma_x, \sigma_y, \sigma_z)$

### 2.2 波函数拟设

波函数拟设（ansatz）是一段含参数量子门的量子线路（Parameterized quantum circuit，PQC），用于表征一个未知的幺正变换，通过改变PQC中参数，我们可以改变幺正变换的作用效果。

例如Hardware efficient ansatz是一种硬件友好型的ansatz，它的两比特门只在近邻比特上出现。在MindQuantum中内置了许多ansatz，例如我们可以通过如下代码来生成一个`HardwareEfficientAnsatz`。

##

In [1]:
from mindquantum.algorithm.nisq.chem import HardwareEfficientAnsatz
from mindquantum.core.gates import RY, RZ, Z
ansatz = HardwareEfficientAnsatz(3, [RY, RZ], Z, [(1, 2), (0, 1)])
ansatz.circuit

q0: ──RY(d0_n0_0)────RZ(d0_n0_1)──────────────●─────────RY(d1_n0_0)────RZ(d1_n0_1)──
                                              │
q1: ──RY(d0_n1_0)────RZ(d0_n1_1)────●─────────Z─────────RY(d1_n1_0)────RZ(d1_n1_1)──
                                    │
q2: ──RY(d0_n2_0)────RZ(d0_n2_1)────Z────RY(d1_n2_0)────RZ(d1_n2_1)─────────────────

如上，ansatz由量子参数门RY、RZ和非参数门CNOT门构成。更多ansatz请参考：

![ansatz](images/ansatz.png)

### 2.3 损失函数

一般来说，我们需要对参数化量子线路演化出的末态进行测量，并计算出测量结果关于某些哈密顿量的期望值，假设所需求解问题的哈密顿量为$H$，量子态拟设Ansatz为$U(\theta)$，则有：

$$E(\theta)=\left<0\right|U^\dagger(\theta)HU(\theta)\left|0\right>$$

对于基态能量求解问题：

$$E_\text{min}=\min_\theta L(\theta)=\min_\theta E(\theta)=\min_\theta \left<0\right|U^\dagger (\theta)HU(\theta)\left|0\right>$$

对于监督问题：

$$\min_\theta L(\theta)=\min_{\theta}L(E(\theta),D_\text{target})$$

在上述问题中，机器学习的损失函数都是适用的。

### 2.4 求导规则

对于一般的损失函数

$$L(E(\theta), D_\text{target})$$

对其求导：

$$\frac{\partial L(E(\theta), D\text{target})}{\partial \theta}=\frac{\partial L}{\partial E}\frac{\partial E}{\partial \theta}$$

因此，我们只需求期望值关于参数的导数：

$$\frac{\partial E}{\partial \theta}=\frac{\partial \left<0\right|U^\dagger (\theta)HU(\theta)\left|0\right>}{\partial \theta}=2\text{Re}\left(
    \left<0\right|U^\dagger(\theta)H\frac{\partial U(\theta)}{\partial \theta}\left|0\right>
\right)$$

由上可发现，对期望值求导可归结为对量子门求导。

含参量子门通常为RX、RY和RZ，我们以RX门为例，求其导数形式。

$$RX(\theta)=e^{-i\theta X/2}=\cos\theta/2 I-i\sin\theta/2 X$$

对$\theta$求导数可得：

$$\frac{\partial RX(\theta)}{\partial \theta}=-iXe^{-i\theta X/2}/2$$

- Parameter Shift rule

对上式进行更进一步运算

$$
\begin{align*}
\frac{\partial RX(\theta)}{\partial \theta}&=-iXe^{-i\theta X/2}/2 \\
&=-\sin(\theta/2)I/2-i\cos(\theta/2)X/2 \\
&=\cos((\theta+\pi)/2)I/2-i\sin((\theta+\pi)/2)X/2 \\
&=\frac{1}{2}RX(\theta+\pi)
\end{align*}
$$

更进一步，我们可以得到parameter shift rule:

$$\frac{\partial E}{\partial\theta}=\frac{1}{2}\left(E(\theta+\pi/2)-E(\theta-\pi/2)\right)$$

### 2.5 优化步骤

- 非梯度优化算法（Gradient Free）：SPSA、Nelder-Mead、Powell、COBYLA等

![non_grad](images/non_grad.png)

- 梯度优化算法（Gradient Base）：BFGS、ADAM、ADGRAD

![grad](images/grad.png)


## 3. 应用领域

[组合优化Max-Cut问题求解教程](https://mindspore.cn/mindquantum/docs/zh-CN/master/quantum_approximate_optimization_algorithm.html)

## 4. 作业
[Max-Cut问题求解作业](https://gitee.com/mindspore/mindquantum/blob/research/meeting/uestc_qc/homework_d4.ipynb)